In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import seaborn as sns

import requests
from bs4 import BeautifulSoup
import re
import time
import os

# 그래프 설정
# plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['font.size'] = 16
plt.rcParams['figure.figsize'] = 20, 10
plt.rcParams['axes.unicode_minus'] = False

### 1. 웹 정보 수집하기

In [2]:
# 주소 (나무위키 최근 변경내역 페이지)
site = 'https://namu.wiki/RecentChanges'

In [3]:
# 요청한다.
res = requests.get(site)
html = res.text
soup = BeautifulSoup(html, 'lxml')

In [4]:
# 데이터를 수집한다.
# 데이터를 가져오고자 하는 부분을 개발자도구를 통해 확인하고 copy 한다.
# 테이블 정보
table_tag = soup.select_one('#app > div > div:nth-child(2) > article > div > table')

# 테이블에 속한 tbody 정보
tbody_tag = table_tag.select_one('tbody')

# tbody안 tr 정보
tr_tag_list = tbody_tag.select('tr')

In [5]:
# 링크 주소들을 담을 리스트
link_list = []


# 위에서 수집한 tr 태그의 수 만큼 반복한다.
for tr_tag in tr_tag_list:
    # tr 태그안에 있는 td 태그들을 가져온다.
    td_list = tr_tag.select('td')
    
    # 추출한 td태그중 첫번째 것만 사용한다.
    td_tag = td_list[0]
    
    # td태그 내의 a태그를 가져온다.
    a_tag_list = td_tag.select('a')
    
    # 가져온 a태그가 있을 경우에만 작업을 한다.
    if len(a_tag_list) > 0:
        # 첫번째 a 태그의 href 속성값을 가져온다.
        href = a_tag_list[0].get('href')
     
        # 리스트에 담는다.
        link_list.append(href)

link_list[:10]

['/w/%EB%B0%B1%EC%8A%B9%ED%98%B8/%ED%81%B4%EB%9F%BD%20%EA%B2%BD%EB%A0%A5',
 '/w/%ED%83%80%EB%AC%B4%EB%9D%BC%20%ED%98%B8%EB%85%B8',
 '/w/%EA%B0%84%EB%A0%88%EC%9D%B4',
 '/w/%EB%8D%94%20%EB%89%B4%20%EB%8D%B0%EC%9D%B4',
 '/w/%EC%8A%A4%EB%A7%A4%EC%8B%9C%EB%B8%8C%EB%9D%BC%EB%8D%94%EC%8A%A4%20%EC%8B%9C%EB%A6%AC%EC%A6%88',
 '/w/%EB%84%A4%EC%9D%B4%EB%B2%84(%EA%B8%B0%EC%97%85)',
 '/w/CADPAT',
 '/w/The%20Catacombs(%ED%95%98%EC%9D%B4%ED%94%BD%EC%85%80/SkyBlock)',
 '/w/CD%20PROJEKT',
 '/w/%EB%B2%A0%EC%9D%B4%EB%B8%94%EB%A0%88%EC%9D%B4%EB%93%9C%20%EB%B2%84%EC%8A%A4%ED%8A%B8/%ED%8C%8C%EC%B8%A0']

In [6]:
# 주소의 수만큼 반복한다.
for sub_url in link_list: 
    time.sleep(1)
    
    # 데이터를 담을 딕셔너리
    data_dict ={
        'title' : [],
        'content' : []
    }
    
    # 페이지 내의 데이터를 수집한다.
    # 요청 주소
    site2 = 'https://namu.wiki' + sub_url
    print(f'{site2} 수집중')
    # 요청한다.
    res2 = requests.get(site2)
    soup2 = BeautifulSoup(res2.text, 'lxml')

    # 글 전체를 가져온다.
    article_tag = soup2.select_one('#app > div > div:nth-child(2) > article')
    # 글 제목을 가져온다
    h1_tag = article_tag.select_one('h1')
    title_text  = h1_tag.text.strip()
    # 글을 가지고 있는 태그들을 가져온다.
    content_paragraphs = article_tag.select('div.wiki-paragraph')

    # 글의 수만큼 반복한다.
    # 글 내용을 하나의 문자열로 합친다.
    content_text = ''
    for content_tag in content_paragraphs:
        content_text = content_text + content_tag.text

    content_text

    # 제거하고자 하는 문자열들을 제거한다.
    content_text = content_text.replace('(','')
    content_text = content_text.replace(')','')
    content_text = content_text.replace('[','')
    content_text = content_text.replace(']','')
    content_text = content_text.strip() # 좌우 공백제거
    
    # print(title_text)
    
    # 데이터를 담는다.
    data_dict['title'].append(title_text)
    data_dict['content'].append(content_text)
    
    # 데이터 프레임 생성
    df1 = pd.DataFrame(data_dict)

    # 생성된 데이터 프레임 저장
    if os.path.exists('wiki.csv') == False :
        # 파일이 없을 경우
        df1.to_csv('wiki.csv', encoding='utf-8-sig', index=False)
    else :
        df1.to_csv('wiki.csv', encoding='utf-8-sig', index=False, header=False, mode='a')

https://namu.wiki/w/%EB%B0%B1%EC%8A%B9%ED%98%B8/%ED%81%B4%EB%9F%BD%20%EA%B2%BD%EB%A0%A5 수집중
https://namu.wiki/w/%ED%83%80%EB%AC%B4%EB%9D%BC%20%ED%98%B8%EB%85%B8 수집중
https://namu.wiki/w/%EA%B0%84%EB%A0%88%EC%9D%B4 수집중
https://namu.wiki/w/%EB%8D%94%20%EB%89%B4%20%EB%8D%B0%EC%9D%B4 수집중
https://namu.wiki/w/%EC%8A%A4%EB%A7%A4%EC%8B%9C%EB%B8%8C%EB%9D%BC%EB%8D%94%EC%8A%A4%20%EC%8B%9C%EB%A6%AC%EC%A6%88 수집중
https://namu.wiki/w/%EB%84%A4%EC%9D%B4%EB%B2%84(%EA%B8%B0%EC%97%85) 수집중
https://namu.wiki/w/CADPAT 수집중
https://namu.wiki/w/The%20Catacombs(%ED%95%98%EC%9D%B4%ED%94%BD%EC%85%80/SkyBlock) 수집중
https://namu.wiki/w/CD%20PROJEKT 수집중
https://namu.wiki/w/%EB%B2%A0%EC%9D%B4%EB%B8%94%EB%A0%88%EC%9D%B4%EB%93%9C%20%EB%B2%84%EC%8A%A4%ED%8A%B8/%ED%8C%8C%EC%B8%A0 수집중
https://namu.wiki/w/C-3PO 수집중
https://namu.wiki/w/%EC%9D%B8%EC%B2%9C%20%EC%A0%84%EC%9E%90%EB%9E%9C%EB%93%9C%20%EC%97%98%EB%A6%AC%ED%8E%80%EC%B8%A0/20-21%EC%8B%9C%EC%A6%8C 수집중
https://namu.wiki/w/%EB%B0%B1%EC%A2%85%EC%9B%90%EC%9D%98%20%EC%9A%94

ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
wiki_df = pd.read_csv('wiki.csv')
wiki_df